In [1]:
from __future__ import print_function, division
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from gensim import corpora, models, similarities, matutils

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import sklearn.metrics.pairwise as smp

/Users/KVASU/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
magic_cards = pd.read_csv("../MagicDatasets/Magic_Pandas_DF")

/Users/KVASU/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
magic_cards = magic_cards.drop(magic_cards.columns[0], axis=1)

In [4]:
magic_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31705 entries, 0 to 31704
Data columns (total 39 columns):
artist               31704 non-null object
border               22 non-null object
cmc                  28226 non-null float64
colorIdentity        26753 non-null object
colors               25061 non-null object
flavor               18801 non-null object
hand                 116 non-null float64
id                   31705 non-null object
imageName            31705 non-null object
layout               31705 non-null object
life                 116 non-null float64
loyalty              158 non-null float64
manaCost             28153 non-null object
manaCostmciNumber    0 non-null float64
mciNumber            22546 non-null object
multiverseid         29487 non-null float64
name                 31705 non-null object
names                351 non-null object
number               25422 non-null object
originalText         29120 non-null object
originalType         29842 non-null obje

# Create vectorized text fields

In [5]:
magic_cards_fill = magic_cards.fillna(" ")
magic_cards_fill['combined_text'] = magic_cards_fill['name']+" "+magic_cards_fill['type']+" "+magic_cards_fill['colors']+" "+magic_cards_fill["text"]+" "+magic_cards_fill["flavor"]+" "+magic_cards_fill["rarity"]
magic_cards_s = magic_cards_fill['combined_text']
magic_list = magic_cards_s.tolist
magic_list = list(magic_cards_s)

In [6]:
tfidf = TfidfVectorizer(stop_words="english", 
                        token_pattern="\\b[a-zA-Z0-9][a-zA-Z0-9]+\\b", 
                        min_df=10)


In [7]:
tfidf_vecs = tfidf.fit_transform(magic_list)


In [8]:
tfidif_df = pd.DataFrame(tfidf_vecs.todense(), 
             columns=tfidf.get_feature_names()
            )

# Other Features

In [9]:
new_cards = magic_cards
new_cards = new_cards.fillna(0.0)

magic_cards_fill_cut = new_cards.iloc[:,[2,11,21,34]]
magic_cards_fill_cut.info()

dummied_df = pd.get_dummies(magic_cards_fill_cut)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31705 entries, 0 to 31704
Data columns (total 4 columns):
cmc          31705 non-null float64
loyalty      31705 non-null float64
power        31705 non-null object
toughness    31705 non-null object
dtypes: float64(2), object(2)
memory usage: 990.9+ KB


In [10]:
new_cards2 = magic_cards
new_cards2 = new_cards2.fillna(0.0)
magic_cards_fill_cut2 = new_cards2.iloc[:,[2,11,21,34]]

new_df = magic_cards_fill_cut2.astype(str)
dummied_df2 = pd.get_dummies(new_df)
dummied_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31705 entries, 0 to 31704
Data columns (total 80 columns):
cmc_0.0          31705 non-null float64
cmc_0.5          31705 non-null float64
cmc_1.0          31705 non-null float64
cmc_10.0         31705 non-null float64
cmc_1000000.0    31705 non-null float64
cmc_11.0         31705 non-null float64
cmc_12.0         31705 non-null float64
cmc_13.0         31705 non-null float64
cmc_14.0         31705 non-null float64
cmc_15.0         31705 non-null float64
cmc_16.0         31705 non-null float64
cmc_2.0          31705 non-null float64
cmc_3.0          31705 non-null float64
cmc_4.0          31705 non-null float64
cmc_5.0          31705 non-null float64
cmc_6.0          31705 non-null float64
cmc_7.0          31705 non-null float64
cmc_8.0          31705 non-null float64
cmc_9.0          31705 non-null float64
loyalty_0.0      31705 non-null float64
loyalty_2.0      31705 non-null float64
loyalty_3.0      31705 non-null float64
loyalty_4.0

# Combine dummied and vectorized df

In [11]:
combined_df = pd.concat([dummied_df2, tfidif_df], axis=1, join_axes=[magic_cards_fill_cut.index])

In [12]:
combined_df.shape

(31705, 4964)

In [13]:
model = NearestNeighbors(n_neighbors=20,n_jobs=-1)

In [14]:
knn = model.fit(combined_df)

/Users/KVASU/anaconda2/lib/python2.7/site-packages/pandas/indexes/base.py:1237: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return key in self._engine


# Setting up Dictionary and other stuff

In [15]:
set_dict = {
    "[10E]" : "Tenth-Edition", "[2ED]" : "Unlimited", "[3ED]" : "Revised", "[4ED]" : "Fourth-Edition",
    "[5DN]" : "Fifth-Dawn", "[5ED]" : "Fifth-Edition", "[6ED]" : "Sixth-Edition", "[7ED]" : "Seventh-Edition",
    "[8ED]" : "Eighth-Edition", "[9ED]" : "Ninth-Edition", "[ALA]" : "Shards-of-Alara", "[ALL]" : "Alliances",
    "[APC]" : "Apocalypse", "[ARB]" : "Alara-Reborn", "[ARC]" : "Archenemy", "[ARN]" : "Arabian-Nights",
    "[ATQ]" : "Aniquities", "[AVR]" : "Avacyn-Restored", "[BFZ]" : "Battle-for-Zendikar", 
    "[BNG]" : "Born-of-the-Gods", "[BOK]" : "Betrayers-of-Kamigawa", "[BRB]" : "Battle-Royale", "[BTB]" : "Beatdown",              "[C13]" : "Commander 2013",
    "[C14]" : "Commander-2014", "[C15]" : "Commander-2015", "[CED]" : "Collectors'-Edition",
    "[CEI]" : "International-Edition", "[CHK]" : "Champions-of-Kamigawa", "[CHR]" : "Chronicles",
    "[CM1]" : "Commander's-Arsenal", "[CMD]" : "Commander", "[C13]" : "Commander-2013",
    "[DD2]" : "Duel Decks: Jace vs. Chandra", "[DD3_DVD]" : "Duel Decks: Divine vs. Demonic", 
    "[DD3_EVG]" : "Duel Decks: Elves vs. Goblins", "[DD3_GVL]" : "Duel Decks: Garruk vs. Liliana",
    "[DD3_JVC]" : "Duel Decks: Jace vs. Chandra", "[DDC]" : "Duel Decks: Divine vs. Demonic",
    "[DDD]" : "Duel Decks: Garruk vs. Liliana", "[DDF]" : "Duel Decks: Elspeth vs. Tezzeret",
    "[DDG]" : "Duel Decks: Knights vs. Dragons", "[DDH]" : "Duel Deck: Ajani vs. Nicol Bolas",
    "[DDI]" : "Duel Decks: Venser vs. Koth", "[DDJ]" : "Duel Decks: Izzet vs. Golgari", 
    "[DDK]" : "Duel Decks: Sorin vs. Tibalt", "[DDL]" : "Duel Decks: Heroes vs. Monsters",
    "[DDO]" : "Duel Decks: Elspeth vs. Kiora", "[DKA]" : "Dark-Ascension", "[FVD]" : "From-the-Vault:-Dragons",
    "[DRK]" : "The-Dark", "[DTK]" : "Dragons-of-Tarkir", "[EVE]" : "Eventide", "[EVG]" : "Duel Decks: Elves vs. Goblins",
    "[EXO]" : "Exodus", "[EXP]" : "Zendikar-Expeditions", "[FEM]" : "Fallen-Empires", "[FRF]" : "Fate-Reforged",
    "[FUT]" : "Future-Sight", "[HML]" : "Homelands", "[HOP]" : "Plancechase", "[ICE]" : "Ice-Age", "[INV]" : "Invasion",
    "[ISD]" : "Innistrad", "[JOU]" : "Journey-into-Nyx", "[KTK]" : "Khans-of-Tarkir", "[LEA]" : "Alpha", 
    "[LEB]" : "Beta", "[LEG]" : "Legends", "[LGN]" : "Legions", "[LRW]" : "Lorwyn", "[M10]" : "Magic-2010", 
    "[M11]" : "Magic-2011", "[M12]" : "Magic-2012", "[M13]" : "Magic-2013", "[M14]" : "Magic-2014", 
    "[M15]" : "Magic-2015", "[MM2]" : "Modern-Masters-2015", "[MMA]" : "Modern-Masters", "[MMQ]" : "Mercadian-Masques",
    "[MRD]" : "Mirrodin", "[NPH]" : "New-Phyrexia", "[ONS]" : "Onslaught", "[ORI]" : "Magic-Origins", 
    "[PC2]" : "Planechase-2012", "[PCY]" : "Prophecy", "[MIR]" : "Mirage", "[MBS]" : "Mirrodin-Besieged",
    "[CN2]" : "Conspiracy-Take-the-Crown", "[THS]" : "Theros", "[GTC]" : "Gatecrash", "[TSP]" : "Time-Spiral",
    "[OGW]" : "Oath-of-the-Gatewatch", "[SOI]" : "Shadows-Over-Innistrad", "[EMN]" : "Eldritch-Moon", 
    "[KLD]" : "Kaladesh", "[PLC]" : "Planar-Chaos", "[RAV]" : "Ravnica"
}

In [16]:
def checkset(set_list):
    #filter sets until i get one in dict
    for i in range(len(set_list)):
        set_name1 = set_list[i]
        set_name2 = "["+set_name1+"]"
        set_name2 = set_name2.replace(" ","")
        try:
            set_name2 = set_dict[set_name2]
            break
        except:
            pass
            
    return set_name2

In [20]:
def predict(name):

    card_index = (magic_cards_fill[magic_cards_fill['name']==name]).index.tolist()
    card_index = card_index[0]

    distances, indices = knn.kneighbors(combined_df.iloc[card_index,:])

    index = indices[0]
    distance = distances[0]

    nearest_list = []
    i=0
    list_dict = []
    for k in range(0,len(index)):
        card_name = magic_cards.iloc[index[k],16]
        if card_name not in nearest_list:
            if i<11:
                card_dict = {}
                nearest_list.append(card_name)
                if i > 0:
                    card_dict['card'] = (str(i) + ".")
                else:
                    card_dict['card'] = ("Original:")
                card_dict['card_name'] = card_name
                card_dict['distance_away'] = distance[k]
                card_dict['cmc'] = str(magic_cards.iloc[index[k],2])
                card_dict['power_toughness'] = str(magic_cards.iloc[index[k],21])+"/"+str(magic_cards.iloc[index[k],34])
                card_dict['cost'] = str(magic_cards.iloc[index[k],12])
                card_dict['type'] = str(magic_cards.iloc[index[k],35])
                card_dict['sets'] = str(magic_cards.iloc[index[k],22])
                card_dict['text'] = str(magic_cards.iloc[index[k],32])
                card_dict['flavor'] = str(magic_cards.iloc[index[k],5])
                
                #setting up link
                card_name2 = card_name.replace(" ", "-").replace(",","").replace("'","").replace(":","")
                set_name = magic_cards.iloc[index[k],22]
                set_list = set_name.replace("[","").replace("]","").split(",")
                #filter sets until i get one in dict
                set_name2 = checkset(set_list)
                card_dict['link'] = "http://www.cardkingdom.com/mtg/"+str(set_name2)+"/"+card_name2
                
                i+=1
                list_dict.append(card_dict)
            else:
                continue
        else:
            continue

    return list_dict

# Test

In [21]:
name = "Avatar of Hope"
card_index = (magic_cards_fill[magic_cards_fill['name']==name]).index.tolist()
card_index = card_index[0]

distances, indices = knn.kneighbors(combined_df.iloc[card_index,:])

index = indices[0]
distance = distances[0]

card_index

/Users/KVASU/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


2686

In [22]:
list_dict = predict(name)
for d in list_dict:
    print ("---------")
    for key in d:
        print (str(key)+' : '+str(d[key]))

/Users/KVASU/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


---------
card_name : Avatar of Hope
distance_away : 0.0
text : If you have 3 or less life, Avatar of Hope costs {6} less to cast.
Flying
Avatar of Hope can block any number of creatures.
cmc : 8.0
power_toughness : 4/9
cost : {6}{W}{W}
link : http://www.cardkingdom.com/mtg/Prophecy/Avatar-of-Hope
sets : [pPRE, PCY, 8ED]
flavor : nan
type : Creature — Avatar
card : Original:
---------
card_name : Filigree Angel
distance_away : 1.98326307295
text : Flying
When Filigree Angel enters the battlefield, you gain 3 life for each artifact you control.
cmc : 8.0
power_toughness : 4/4
cost : {5}{W}{W}{U}
link : http://www.cardkingdom.com/mtg/Alara-Reborn/Filigree-Angel
sets : [ARB, C13]
flavor : "I craved enlightenment, and Crucius's etherium opened my eyes. I would share my sight with you, but first you must believe."
type : Artifact Creature — Angel
card : 1.
---------
card_name : Guardian of Vitu-Ghazi
distance_away : 1.98388081337
text : Convoke (Your creatures can help cast this spell. Each